查看有哪些國家

In [36]:
r = requests.get('https://spotifycharts.com/regional/tw/weekly/2020-11-13--2020-11-20')
soup = BeautifulSoup(r.text, 'html.parser')
len(soup.find('div', {'class': 'chart-filters-list'}).find('div', {'data-type':'country'}).find('ul').findAll('li'))
countries = [(c['data-value'], c.text) for c in soup.find('div', {'class': 'chart-filters-list'}).find('div', {'data-type':'country'}).find('ul').findAll('li')]
countries

[('global', 'Global'),
 ('us', 'United States'),
 ('gb', 'United Kingdom'),
 ('ad', 'Andorra'),
 ('ar', 'Argentina'),
 ('at', 'Austria'),
 ('au', 'Australia'),
 ('be', 'Belgium'),
 ('bg', 'Bulgaria'),
 ('bo', 'Bolivia'),
 ('br', 'Brazil'),
 ('ca', 'Canada'),
 ('ch', 'Switzerland'),
 ('cl', 'Chile'),
 ('co', 'Colombia'),
 ('cr', 'Costa Rica'),
 ('cy', 'Cyprus'),
 ('cz', 'Czech Republic'),
 ('de', 'Germany'),
 ('dk', 'Denmark'),
 ('do', 'Dominican Republic'),
 ('ec', 'Ecuador'),
 ('ee', 'Estonia'),
 ('es', 'Spain'),
 ('fi', 'Finland'),
 ('fr', 'France'),
 ('gr', 'Greece'),
 ('gt', 'Guatemala'),
 ('hk', 'Hong Kong'),
 ('hn', 'Honduras'),
 ('hu', 'Hungary'),
 ('id', 'Indonesia'),
 ('ie', 'Ireland'),
 ('il', 'Israel'),
 ('in', 'India'),
 ('is', 'Iceland'),
 ('it', 'Italy'),
 ('jp', 'Japan'),
 ('lt', 'Lithuania'),
 ('lu', 'Luxembourg'),
 ('lv', 'Latvia'),
 ('mx', 'Mexico'),
 ('my', 'Malaysia'),
 ('ni', 'Nicaragua'),
 ('nl', 'Netherlands'),
 ('no', 'Norway'),
 ('nz', 'New Zealand'),
 ('pa', '

查看總共有幾週

In [2]:
import requests
from bs4 import BeautifulSoup
r = requests.get('https://spotifycharts.com/regional/tw/weekly/2020-11-13--2020-11-20')
soup = BeautifulSoup(r.text, 'html.parser')
len(soup.find('div', {'class': 'chart-filters-list'}).find('div', {'data-type':'date'}).find('ul').findAll('li'))

205

In [24]:
import requests
from bs4 import BeautifulSoup
import datetime as dt
import pandas as pd

# 要爬的網址、api
spotify = 'https://spotifycharts.com/'
track_api = 'https://api.spotify.com/v1/tracks'
feature_api = 'https://api.spotify.com/v1/audio-features'
artists_api = 'https://api.spotify.com/v1/artists'

# 設定要爬的國家
# regions = [('de', 'Germany'), ('it', 'Italy'), ('es', 'Spain'), ('nl', 'Netherlands'), ('fr', 'France'), ('fi', 'Finland')]
regions = [ ('us', 'United States'), ('gb', 'United Kingdom'), ('at', 'Austria'), ('au', 'Australia'), ('br', 'Brazil')]
for region, country in regions:
    
    time_start = dt.datetime.now()
    print('===================================================')
    print(f'{country} start!!')
    
    '''
    爬每週 top200 的名單
    '''
    fri = dt.date(2020, 11, 27)
    track_ids, w = [], 0
    while w < 205:  # 總共有205週可以爬
        end, start = str(fri - dt.timedelta(days = 7*w)), str(fri - dt.timedelta(days = 7*(w+1)))
        week = f'{start}--{end}'
        w += 1  

        # 爬當週的top200名單
        url = f'{spotify}regional/{region}/weekly/{week}/download' 
        r = requests.get(url)
        table = r.text.split('\n')[2:-1]
        
        # 整理 top200 track_ids
        _ = [track.split(',')[-1].split('/')[-1] for track in table]
        track_ids += _
        print(f'取得week{str(w).zfill(3)} id')
    
    track_ids = list(set(track_ids)) # 只取不重複的歌
    print(f'{country}共有{len(track_ids)}首歌')
    
    '''
    爬 track_api 和 feature_api
    '''
    
    CLIENT_ID = 'ad0d8699855a4e72b183657dc0f7d55a'
    CLIENT_SECRET = '3e66a60843e34360add3d4246c34fc52'
    AUTH_URL = 'https://accounts.spotify.com/api/token'
    auth_response = requests.post(AUTH_URL, {'grant_type': 'client_credentials', 'client_id': CLIENT_ID, 'client_secret': CLIENT_SECRET,})
    # 獲得 access_token (時間到會失效，每爬一個國家就重取一次token)
    access_token = auth_response.json()['access_token']
    headers = {'Authorization': f'Bearer {access_token}'}
    
    data = pd.DataFrame(columns = ['track', 'feature'])
    track, feature = [], []
    i = 0 
    while i < len(track_ids):
        ids_t = ','.join(track_ids[i : i+50])  # tracks 一次最多50筆
        track += requests.get(track_api, params = {'ids':ids_t}, headers = headers).json()['tracks']
        
        if i%100 == 0:
            ids_f = ','.join(track_ids[i:i+100])  # audio-features 一次最多100筆
            feature += requests.get(feature_api, params = {'ids':ids_f}, headers = headers).json()['audio_features']
        
        i += 50
        
    data = pd.concat([data, pd.DataFrame(zip(track, feature), columns = ['track', 'feature'])], axis = 0)

    '''
    把爬蟲爬到的東西分到各個column裡
    '''
    # track_api 爬到的東西
    _ = [('track_name', 'name'), ('track_id', 'id'), ('popularity', 'popularity')]
    for col, fea in _:
        data[col] = data['track'].apply(lambda x: x[fea])
    _ = [('album_name', 'name'), ('album_id', 'id'), ('release_date', 'release_date')] 
    for col, fea in _:
        data[col] = data['track'].apply(lambda x: x['album'][fea])
    _ = [('artist_name', 'name'), ('artist_id', 'id')]    
    for col, fea in _:
        data[col] = data['track'].apply(lambda x: x['artists'][0][fea])

    # feature_api 爬到的東西    
    features = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature']
    for key in features:
        data[key] = data['feature'].apply(lambda x:x[key])
        
    print('取得track_api、feature_api資料')    

    '''
    從 artists_api 取得 genres 資訊
    '''    
    # 建立 artists_id、genres 對照表
    artist_genres = dict()
    artists_ids = list(data['artist_id'].unique())

    i = 0
    while i < len(artists_ids):
        ids = ','.join(artists_ids[i:i+50]) # artists 一次最多50筆

        r = requests.get(artists_api, params = {'ids':ids}, headers = headers)
        artist_genres.update({artist['id']:artist['genres'] for artist in r.json()['artists']})

        i += 50 

    # 取得 genres
    data['genres'] = data['artist_id'].map(artist_genres)
    print('取得genre資料')
    
    # 加上 country 資訊
    data['country'] = country

    print(dt.datetime.now() - time_start)

    data = data.drop(['track', 'feature'], axis = 1)
    data.to_csv(f'/Users/shao/learning_python/ccClub advance/mid_project/data/data_{country}.csv', index = False)


Australia start!!
取得week001 id
取得week002 id
取得week003 id
取得week004 id
取得week005 id
取得week006 id
取得week007 id
取得week008 id
取得week009 id
取得week010 id
取得week011 id
取得week012 id
取得week013 id
取得week014 id
取得week015 id
取得week016 id
取得week017 id
取得week018 id
取得week019 id
取得week020 id
取得week021 id
取得week022 id
取得week023 id
取得week024 id
取得week025 id
取得week026 id
取得week027 id
取得week028 id
取得week029 id
取得week030 id
取得week031 id
取得week032 id
取得week033 id
取得week034 id
取得week035 id
取得week036 id
取得week037 id
取得week038 id
取得week039 id
取得week040 id
取得week041 id
取得week042 id
取得week043 id
取得week044 id
取得week045 id
取得week046 id
取得week047 id
取得week048 id
取得week049 id
取得week050 id
取得week051 id
取得week052 id
取得week053 id
取得week054 id
取得week055 id
取得week056 id
取得week057 id
取得week058 id
取得week059 id
取得week060 id
取得week061 id
取得week062 id
取得week063 id
取得week064 id
取得week065 id
取得week066 id
取得week067 id
取得week068 id
取得week069 id
取得week070 id
取得week071 id
取得week072 id
取得week073 id
取得week074 id
取得week075 id
取得week0